Метод в лоб

In [ ]:
import tkinter as tk
from tkinter import ttk


PEAK_HOURS = [(7, 9), (17, 19)]
SHIFT_DURATION = 9
BREAK_DURATION_BIG = 1
BREAK_DURATION_SMALL = 0.25
BREAK_INTERVAL = 3


drivers = [
    {"name": "Водитель 1", "start": 6, "type": "A"},
    {"name": "Водитель 2", "start": 7, "type": "A"},
    {"name": "Водитель 3", "start": 8, "type": "B"},
    {"name": "Водитель 4", "start": 10, "type": "A"},
    {"name": "Водитель 5", "start": 11, "type": "B"},
    {"name": "Водитель 6", "start": 13, "type": "B"},
    {"name": "Водитель 7", "start": 16, "type": "A"},
    {"name": "Водитель 8", "start": 18, "type": "B"},
]


def is_peak(hour):
    for start, end in PEAK_HOURS:
        if start <= hour < end:
            return True
    return False


def assign_breaks(shift_hours, driver_type):
    breaks = []
    possible_breaks = [hour for hour in shift_hours if not is_peak(hour)]

    if driver_type == "A":
        for i, hour in enumerate(shift_hours):
            if hour in possible_breaks and i >= 4:
                breaks.append(hour)
                break
    elif driver_type == "B":
        first_break = None
        second_break = None
        for i, hour in enumerate(shift_hours):
            if hour in possible_breaks and i >= 2:
                if first_break is None:
                    first_break = hour
                elif (hour - first_break) % 24 >= BREAK_INTERVAL:
                    second_break = hour
                    break
        if first_break is not None:
            breaks.append(first_break)
        if second_break is not None:
            breaks.append(second_break)
    return breaks


def create_driver_schedule(driver):
    schedule = {}
    shift_start = driver['start']
    shift_end = (shift_start + SHIFT_DURATION) % 24
    shift_hours = [(shift_start + i) % 24 for i in range(SHIFT_DURATION)]

    breaks = assign_breaks(shift_hours, driver['type'])

    for hour in shift_hours:
        if hour == shift_start:
            if is_peak(hour):
                schedule[hour] = 'Начало смены (час пик)'
            else:
                schedule[hour] = 'Начало смены'
        elif hour in breaks:
            schedule[hour] = 'Перерыв'
        elif is_peak(hour):
            schedule[hour] = 'Поездка (час пик)'
        else:
            schedule[hour] = 'Поездка'

    schedule[shift_end] = 'Окончание смены'
    return schedule


def generate_schedule():
    driver_schedules = {}
    for driver in drivers:
        driver_schedules[driver['name']] = create_driver_schedule(driver)
    return driver_schedules


def create_gui(schedule):
    root = tk.Tk()
    root.title("Расписание водителей автобусов")
    root.geometry("1800x700")

    canvas = tk.Canvas(root)
    scrollbar = ttk.Scrollbar(root, orient="horizontal", command=canvas.xview)
    scrollable_frame = ttk.Frame(canvas)

    scrollable_frame.bind(
        "<Configure>",
        lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )

    canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
    canvas.configure(xscrollcommand=scrollbar.set)

    canvas.pack(side="top", fill="both", expand=True)
    scrollbar.pack(side="bottom", fill="x")

    for idx, (driver, hours) in enumerate(schedule.items()):
        frame = ttk.LabelFrame(scrollable_frame, text=driver, padding=10)
        frame.grid(row=0, column=idx, padx=10, pady=5, sticky="n")

        hours_sorted = sorted(hours.items())
        for hour, activity in hours_sorted:
            label = ttk.Label(frame, text=f"{hour:02d}:00 - {activity}", anchor="w", padding=5)
            label.pack(fill="x")

    root.mainloop()


if __name__ == "__main__":
    schedule = generate_schedule()
    create_gui(schedule)
